In [1]:
import pandas as pd
import json

In [2]:
with open ('safecity_crime_data.json','r') as file:
    data=json.load(file)

In [3]:
del data['status']
del data['message']
del data['query']
del data['count']

In [4]:
data=data['data']


In [5]:
coordinates = pd.DataFrame(data)

In [6]:
coordinates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13668 entries, 0 to 13667
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         13668 non-null  object
 1   latitude   13668 non-null  object
 2   longitude  13668 non-null  object
dtypes: object(3)
memory usage: 320.5+ KB


In [7]:
with open('safecity_incident_details.json','r') as file:
    incident_details=json.load(file)
    

In [8]:
for incident in incident_details:
    del incident['status']
    del incident['message']

In [9]:
details=[]
for incident in incident_details:
    if 'data' in incident:
        details.append(incident['data'])

In [10]:
for incident in details:
    del incident['answers']
    del incident['posted_by']
    del incident['detail_id']
    del incident['answer_tag']
    del incident['cat_ids']
    del incident['admin_id']
    del incident['status']
    del incident['building']

In [11]:
incident_details=pd.DataFrame(details)

In [12]:
categories=incident_details['categories'].unique()

In [13]:

crime_severity = {
    # Minor harassment (1-3)
    'Ogling/Facial Expressions/Staring': 2,
    
    # Moderate crimes (4-6)
    'Taking photos without permission': 4,
    'Showing Pornography without consent': 5,
    'Online Harassment': 4,
    'Chain Snatching/Robbery': 5,
    'Stalking': 6,
    
    # Serious crimes (7-8)
    'Commenting/Sexual Invites': 7,  
    'Touching /Groping': 7,
    'Indecent Exposure/Masturbation in public': 7,
    'Physical assault': 8,
    
    # Severe crimes (9-10)
    'Rape/Sexual Assault': 10,
    'Human Trafficking': 10,
    'Domestic Violence': 9,
    
    # Catch-all
    'Others': 3  # Default moderate rating for unspecified incidents
}

def calculate_severity(category_string):
    """
    Calculate severity rating for a category string containing multiple crimes
    separated by ' | '. Returns the highest severity rating found.
    """
    if not category_string or category_string.strip() == '':
        return 1  # Empty category gets lowest rating
    
    # Split by ' | ' to get individual crimes
    crimes = [crime.strip() for crime in category_string.split(' | ')]
    
    # Find the maximum severity rating
    max_severity = 1
    for crime in crimes:
        if crime in crime_severity:
            max_severity = max(max_severity, crime_severity[crime])
    
    return max_severity

# # Apply severity calculation to your categories
# category_severities = []
# for category in categories:
#     severity = calculate_severity(category)
#     category_severities.append({
#         'category': category,
#         'severity': severity
#     })

# # Print summary of updated ratings
# print("Updated Crime Severity Distribution:")
# for item in category_severities[:10]:  # Show first 10 examples
#     print(f"'{item['category']}' -> Severity: {item['severity']}")


def get_severity_rating(category_string):
    """Simple function to get severity rating for a category string"""
    return calculate_severity(category_string)

In [14]:
incident_details['severity'] = incident_details['categories'].apply(get_severity_rating)

In [15]:
incident_details.drop(['lang_id','landmark','is_date_estimate','is_time_estimate','additional_detail'], axis=1, inplace=True)

In [16]:
incident_details.head()

,id,area,city,state,country,latitude,longitude,created_on,description,age,gender_id,gender,incident_date,time_from,time_to,categories,severity
0,11837,Nehru Place,New Delhi,Delhi,India,28.551478,77.251543,2020-12-01 19:45:28,Sexual Invite : A man walked with me for 5 min...,0,0,None,2012-12-25,01:50:00,00:00:00,Commenting/Sexual Invites,7
1,11842,Block G,New Delhi,Delhi,India,28.568814,77.220565,2020-12-01 19:45:28,Sexual Invite : Late evening (around 7.30 pm) ...,0,0,None,2012-12-26,01:56:00,00:00:00,Commenting/Sexual Invites,7
2,11844,Ajmeri Gate,New Delhi,Delhi,India,28.641970,77.220600,2020-12-01 19:45:28,"Pitampura, Bus stand : Comments were passed wh...",0,0,None,2012-12-26,17:57:00,00:00:00,Commenting/Sexual Invites,7
3,11848,Kalkaji Extension,New Delhi,Delhi,India,28.542735,77.259194,2020-12-01 19:45:28,Retard commenting : I just stepped out of offi...,0,0,None,2012-12-27,01:49:00,00:00:00,Commenting/Sexual Invites,7
4,11851,Ajmeri Gate,New Delhi,Delhi,India,28.641970,77.220600,2020-12-01 19:45:28,New Delhi Railway Station Indecent comments : ...,0,0,None,2009-06-15,11:07:00,00:00:00,Commenting/Sexual Invites,7


In [17]:
incident_details.columns

Index(['id', 'area', 'city', 'state', 'country', 'latitude', 'longitude',
       'created_on', 'description', 'age', 'gender_id', 'gender',
       'incident_date', 'time_from', 'time_to', 'categories', 'severity'],
      dtype='object')

In [18]:
# incident_details.drop(['id'], axis=1, inplace=True)

In [19]:
# incident_details.to_csv("processed_incident_details.csv", index=False)